In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
dfw = pd.read_parquet("../inference/dfw_big.parquet")
dfl = pd.read_parquet("../inference/dfl_big.parquet")
len(dfw), len(dfl)

In [ ]:
dfw.head(2)

In [ ]:
dfl.head(2)

In [ ]:
# lo.loc[(lo.system == 2) & (lo.datetime == lo.iloc[0].datetime)]

In [ ]:
ax = sns.lineplot(
    data=dfl,
    x="step",
    y="abs_err",
    hue="system",
)
ax.set_title("abs_err")

In [ ]:
ax = sns.lineplot(
    data=dfl,
    x="step",
    y="squ_err",
    hue="system",
)
ax.set_title("squ_err")